# Map and Filter Group Work

In this exercise, we will revisit the group work assignment on reading and writing tables.  Below you will find the answer key for the earlier assignment.  Your task will be to convert all code to use `map` and `filter` whereever possible.

**Here are the rules**

1. Use `with_iter` from `more_itertools` combined with `csv.reader` to read tables.
2. Replace all comprehension with calls to `map` and/or `filter`
3. Write `lambda` helper functions for each `map` and `filter` 



<font color="red">**Question 1:**</font>  While rare in the modern era, it used to be common practice for teams to have player-managers, i.e. using a current player as a manager.  Compute the total number of runs scored by all player-managers (all time).

**Relevant Files:**  Managers.csv, Batting.csv

**Hint:** You will need to join the tables on `playerID` and `year`

**Step 1:** Import `with_iter` from `more_itertools`, `get` from `toolz`, and `reader` from `csv`

In [1]:
from more_itertools import with_iter
from csv import reader
from toolz import get

** Step 2:** Read *Manager.csv* and *Batting.csv* into Python

In [2]:
# Note the replicated code.  We will fix this in a future exercise
read_csv = lambda filename: list(reader(with_iter(open(filename))))
managers = read_csv("Managers.csv")
batting = read_csv("Batting.csv")

**Step 3:** Inspect the header of managers using `list` and `enumerate`.

In [3]:
managers_header = managers[0]
list(enumerate(managers_header))

[(0, 'playerID'),
 (1, 'yearID'),
 (2, 'teamID'),
 (3, 'lgID'),
 (4, 'inseason'),
 (5, 'G'),
 (6, 'W'),
 (7, 'L'),
 (8, 'rank'),
 (9, 'plyrMgr')]

**Step 4:** Filter out the player managers into a new file

In [4]:
player_managers = [r for r in managers if get(-1, r) == 'Y']

**Step 5:** Make a list of player managers IDs 

In [5]:
player_managers_IDs = [get(0, r) for r in player_managers]

**Step 6:** Inspect the batting header using `list` and `enumerate`.

In [6]:
batting_header = batting[0]
list(enumerate(batting_header))

[(0, 'playerID'),
 (1, 'yearID'),
 (2, 'stint'),
 (3, 'teamID'),
 (4, 'lgID'),
 (5, 'G'),
 (6, 'AB'),
 (7, 'R'),
 (8, 'H'),
 (9, '2B'),
 (10, '3B'),
 (11, 'HR'),
 (12, 'RBI'),
 (13, 'SB'),
 (14, 'CS'),
 (15, 'BB'),
 (16, 'SO'),
 (17, 'IBB'),
 (18, 'HBP'),
 (19, 'SH'),
 (20, 'SF'),
 (21, 'GIDP')]

**Step 7:** Filter out the runs from *Batting.csv* for all player managers

In [7]:
runs = [int(get(7, r)) for r in batting if get(0,r) in player_managers_IDs]

**Step 8:** Total the runs

In [8]:
sum(runs)

159530

<font color="red">**Question 2:**</font> Create a table of statistics on pitching that contains the following information:

* players_name, 
* year, 
* team_name, (**BONUS**)
* wins(W), 
* losses(L), 
* complete games (CG), 
* innings pitched (IP)

Write this information to a file named **PitchingNames.csv** and include this file with your submission.

**Edit:** Do this for the last 10 years.

**Bonus:** Do this for all time.

**Hint:** You can speed this up by filtering the master table, keeping only rows related to pitchers.

**Relevant Files:** Pitching.csv, Master.csv, Teams.csv

In [21]:
pitching = read_csv('Pitching.csv')
master = read_csv('Master.csv')
teams = read_csv('Teams.csv')

In [22]:
# Step 1: get the info from pitching.
# We need playerID (0), yearId(1), teamID (3),wins (5), loses(6), complete games (9) and IPouts (12) 
# See enumeration in Question 3
filter_row = lambda r: get([0, 1, 3, 5, 6, 9, 12], r)
pitching2006_2010 = [filter_row(r) for r in pitching[1:] if 2006 <= int(get(1, r)) <= 2010]

In [23]:
# To get the players names, we need a list of all players in the filtered list
pitchersIDs = [get(0, r) for r in pitching2006_2010]

In [24]:
# Now filter out the names from master.
# We need the playerID(0), first name(13) and last name(14).  See the enumeration below.
raw_names = [get([0, 13, 14], r) for r in master if get(0, r) in pitchersIDs]
# Now process these names by adding the first and last names.  Do this with a helper function.
add_names = lambda row: (row[0], row[1] + " " + row[2])
names = [add_names(r) for r in raw_names]
names[:3]

[('aardsda01', 'David Aardsma'),
 ('abadfe01', 'Fernando Abad'),
 ('abreuwi01', 'Winston Abreu')]

In [25]:
# Time to join the names onto the pitcher info.  
# We want to keep the second index from the names and 
# indexes [1,3,4,6,9,12] from piching info.
# Make a helper function to do this
combined_row = lambda name_row, info_row: get([1], name_row) + get([1, 2, 3, 4, 5, 6], info_row)
# The rows will be joined by mathcing playerIDs.  Here is a helper to do this.
match_on_playerID = lambda name_row, info_row: get(0, name_row) == get(0, info_row)
info_with_names = [combined_row(name_row, info_row) 
                   for name_row in names
                  for info_row in pitching2006_2010
                  if match_on_playerID(name_row, info_row)]
info_with_names[:3]

[('David Aardsma', '2006', 'CHN', '3', '0', '0', '159'),
 ('David Aardsma', '2007', 'CHA', '2', '1', '0', '97'),
 ('David Aardsma', '2008', 'BOS', '4', '2', '0', '146')]

In [26]:
# Do the same for the teams.
# 1. Pull out the teamIDs for filtering.
# 2. Make a table with teamID and team name
# 3. Join the tables, replacing teamID with team name (for convenience, we will put the team name on the end.)
teamIDs = [get([1, 2], r) for r in info_with_names]
teamIDs[:1]

[('2006', 'CHN')]

In [27]:
team_names = [get([0,2,40], r) for r in teams if get([0,2],r) in teamIDs]
team_names[:3]

[('2006', 'ARI', 'Arizona Diamondbacks'),
 ('2006', 'ATL', 'Atlanta Braves'),
 ('2006', 'BAL', 'Baltimore Orioles')]

In [28]:
combined_row = lambda team_row, info_row: get([0,1], info_row) + get([2], team_row) + get([3, 4, 5, 6], info_row)
join_on_teamID_year  = lambda team_row, info_row: get(0, team_row) == get(1, info_row) and get(1, team_row) == get(2, info_row)
info_with_name_team = [combined_row(team_row, info_row)
                      for team_row in team_names
                      for info_row in info_with_names
                      if join_on_teamID_year(team_row, info_row)]
info_with_name_team[:3]

[('Greg Aquino', '2006', 'Arizona Diamondbacks', '2', '0', '0', '145'),
 ('Jeff Bajenaru', '2006', 'Arizona Diamondbacks', '0', '1', '0', '3'),
 ('Miguel Batista', '2006', 'Arizona Diamondbacks', '11', '8', '3', '619')]

In [29]:
from more_itertools import consume
# Join the rows to strings then write the files.O
joined_rows = [",".join(r) for r in info_with_name_team]
with open('PitchingNames.csv', 'w') as out:
    consume([print(r, file=out) for r in joined_rows])
! cat PitchingNames.csv | head -n 5

Greg Aquino,2006,Arizona Diamondbacks,2,0,0,145
Jeff Bajenaru,2006,Arizona Diamondbacks,0,1,0,3
Miguel Batista,2006,Arizona Diamondbacks,11,8,3,619
Randy Choate,2006,Arizona Diamondbacks,0,1,0,48
Juan Cruz,2006,Arizona Diamondbacks,5,6,0,284
cat: stdout: Broken pipe


<font color="red">**Question 3:**</font> Find the name of the pitcher and the associated team name for the pitcher with the lowest ERA (earned runs average) out of all pitchers between the years 1970 and 2010.  Limit your search to pitchers with at least 5 games pitched in a given year.

** Relevant Files:** Pitching.csv, Master.csv, Teams.csv

**Step 1:** Read in all three files.

In [30]:
pitching = read_csv('Pitching.csv')
master = read_csv('Master.csv')
teams = read_csv('Teams.csv')

**Step 2:** Filter the pitching table to only include the years 1970 to 2010.

In [31]:
# Look at the pitching header to determine the location of year
pitching_header = pitching[0]
list(enumerate(pitching_header))

[(0, 'playerID'),
 (1, 'yearID'),
 (2, 'stint'),
 (3, 'teamID'),
 (4, 'lgID'),
 (5, 'W'),
 (6, 'L'),
 (7, 'G'),
 (8, 'GS'),
 (9, 'CG'),
 (10, 'SHO'),
 (11, 'SV'),
 (12, 'IPouts'),
 (13, 'H'),
 (14, 'ER'),
 (15, 'HR'),
 (16, 'BB'),
 (17, 'SO'),
 (18, 'BAOpp'),
 (19, 'ERA'),
 (20, 'IBB'),
 (21, 'WP'),
 (22, 'HBP'),
 (23, 'BK'),
 (24, 'BFP'),
 (25, 'GF'),
 (26, 'R'),
 (27, 'SH'),
 (28, 'SF'),
 (29, 'GIDP')]

In [32]:
pitching1970_2010 = [r for r in pitching[1:] if 1970 <= int(get(1, r)) <= 2010]
pitching1970_2010[:1]

[['abernte02',
  '1970',
  '1',
  'CHN',
  'NL',
  '0',
  '0',
  '11',
  '0',
  '0',
  '0',
  '1',
  '27',
  '9',
  '2',
  '0',
  '5',
  '2',
  '0.28',
  '2',
  '1',
  '0',
  '1',
  '0',
  '40',
  '3',
  '2',
  '',
  '',
  '']]

**Step 3:** Make a new table that contains the ERA (int) and playerID of each pitcher in the filtered list

**Note** We also need to hang onto the year and teamID for later joins.

In [33]:
# Pull out the raw string entries using get
eras = [get([0,1,3,18], r) for r in pitching1970_2010]
# Map float to era, making sure we skip empty rows
eras = [(id, year, teamID, float(era)) for id, year, teamID, era in eras if len(era) > 0]
# NOTE we will use an abstraction to allow us to do this in one step in a later exercise
eras[:3]

[('abernte02', '1970', 'CHN', 0.28),
 ('abernte02', '1970', 'KCA', 0.2),
 ('abernte02', '1970', 'SLN', 0.24)]

**Step 4:** Use `min` with a key function (see **Section 4.3.4** of the book) to identify the player with the lowest ERA

In [34]:
# We are sorting by era, which has index 2
key_func = lambda r: get(3, r)
min_era = min(eras, key=key_func)
min_era

('otoolde01', '1971', 'CHA', 0.0)

In [35]:
min_player, min_year, min_team, min_era = min_era
min_player

'otoolde01'

In [36]:
# BONUS - For fun lets find all players with that tie with this era
ties = [r for r in eras if get(3, r) == min_era]
len(ties)

771

In [37]:
ties[:4]

[('otoolde01', '1971', 'CHA', 0.0),
 ('yountla01', '1971', 'HOU', 0.0),
 ('kilkemi01', '1972', 'OAK', 0.0),
 ('apodabo01', '1973', 'NYN', 0.0)]

**Step 5:** Use filters to get the name and the team of the player identified in the last step.

In [38]:
# Use enumerate on the header to find the indexes for nameGiven
master_header = master[0]
list(enumerate(master_header))

[(0, 'playerID'),
 (1, 'birthYear'),
 (2, 'birthMonth'),
 (3, 'birthDay'),
 (4, 'birthCountry'),
 (5, 'birthState'),
 (6, 'birthCity'),
 (7, 'deathYear'),
 (8, 'deathMonth'),
 (9, 'deathDay'),
 (10, 'deathCountry'),
 (11, 'deathState'),
 (12, 'deathCity'),
 (13, 'nameFirst'),
 (14, 'nameLast'),
 (15, 'nameGiven'),
 (16, 'weight'),
 (17, 'height'),
 (18, 'bats'),
 (19, 'throws'),
 (20, 'debut'),
 (21, 'finalGame'),
 (22, 'retroID'),
 (23, 'bbrefID')]

In [39]:
# Get the  name by matching with id
# Note that the name sill be first + last
add_name = lambda row: row[0] + " " + row[1]
player_name = get(0, [add_name(get([13, 14], r)) for r in master if get(0, r) == min_player])
player_name

"Denny O'Toole"

In [40]:
team_header = teams[0]
list(enumerate(team_header))

[(0, 'yearID'),
 (1, 'lgID'),
 (2, 'teamID'),
 (3, 'franchID'),
 (4, 'divID'),
 (5, 'Rank'),
 (6, 'G'),
 (7, 'Ghome'),
 (8, 'W'),
 (9, 'L'),
 (10, 'DivWin'),
 (11, 'WCWin'),
 (12, 'LgWin'),
 (13, 'WSWin'),
 (14, 'R'),
 (15, 'AB'),
 (16, 'H'),
 (17, '2B'),
 (18, '3B'),
 (19, 'HR'),
 (20, 'BB'),
 (21, 'SO'),
 (22, 'SB'),
 (23, 'CS'),
 (24, 'HBP'),
 (25, 'SF'),
 (26, 'RA'),
 (27, 'ER'),
 (28, 'ERA'),
 (29, 'CG'),
 (30, 'SHO'),
 (31, 'SV'),
 (32, 'IPouts'),
 (33, 'HA'),
 (34, 'HRA'),
 (35, 'BBA'),
 (36, 'SOA'),
 (37, 'E'),
 (38, 'DP'),
 (39, 'FP'),
 (40, 'name'),
 (41, 'park'),
 (42, 'attendance'),
 (43, 'BPF'),
 (44, 'PPF'),
 (45, 'teamIDBR'),
 (46, 'teamIDlahman45'),
 (47, 'teamIDretro')]

In [41]:
team_name = get(0, [get(40, r) for r in teams if get(0, r) == min_year and get(2, r) == min_team])
team_name

'Chicago White Sox'

In [42]:
player_name, team_name, min_year, min_era

("Denny O'Toole", 'Chicago White Sox', '1971', 0.0)

In [ ]:
# Bonus exercise: Can you get all the player and team names for the ties?